In [1]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [2]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """

    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    module = np.linalg.norm(vec)
    y = np.zeros_like(vec)
    y[0] = module
    u = np.zeros_like(vec)
    u = (vec-y)/np.linalg.norm(vec-y)
    if abs(u[0])<1e-6:
        u[0] = -(sum(vec**2)-vec[0]**2)/(vec[0]+module) 
    U = u.T
    H = np.identity(np.shape(vec)[0]) - 2 * np.dot(u.reshape(-1,1), U.reshape(1,-1))
    
        
    return y, H

Test your function using tests below:

In [3]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)
print('v',v, 'v1 = h @ v1', h@v1,'v1', v1,'v1 = h @ v',np.dot(h, v),'h', h, sep = '\n')

assert_allclose(np.dot(h, v1), v, atol=1e-15)
assert_allclose(np.dot(h, v), v1, atol=1e-15)

v
[1 2 3]
v1 = h @ v1
[1. 2. 3.]
v1
[3.74165739 0.         0.        ]
v1 = h @ v
[ 3.74165739e+00  0.00000000e+00 -1.11022302e-16]
h
[[ 0.26726124  0.53452248  0.80178373]
 [ 0.53452248  0.61007346 -0.5848898 ]
 [ 0.80178373 -0.5848898   0.12266529]]


In [4]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)
assert_allclose(np.dot(h, v1), vec)
print('vec',vec, 'v1 = h @ v1', h@v1,'v1', v1,'v1 = h @ vec',np.dot(h, vec), sep = '\n')


vec
[0.19151945 0.62210877 0.43772774 0.78535858 0.77997581 0.27259261
 0.27646426]
v1 = h @ v1
[0.19151945 0.62210877 0.43772774 0.78535858 0.77997581 0.27259261
 0.27646426]
v1
[1.4110968 0.        0.        0.        0.        0.        0.       ]
v1 = h @ vec
[ 1.41109680e+00 -1.11022302e-16  2.77555756e-17  5.55111512e-17
  4.16333634e-17  4.51028104e-17  0.00000000e+00]


# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [5]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    H = np.eye(m)
   
    for i in range(n if n<m else m-1):
        matrix = a1[i:, i:]
        vec = matrix[:,0]
        Hi = np.eye(m)
        Hi[i:, i:] = householder(vec)[1]
        a1 = Hi @ a1
        H = Hi @ H
    return H.T, H@a
    

In [6]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc
np.set_printoptions(suppress=True)

In [7]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5,3))
print('a', a, sep='\n')
q, r = qr_decomp(a)
print('q', q, 'r',r, sep='\n')
# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

a
[[0.19151945 0.62210877 0.43772774]
 [0.78535858 0.77997581 0.27259261]
 [0.27646426 0.80187218 0.95813935]
 [0.87593263 0.35781727 0.50099513]
 [0.68346294 0.71270203 0.37025075]]
q
[[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]]
r
[[1.40152769 1.2514379  0.89523615]
 [0.         0.84158252 0.68447942]
 [0.         0.         0.5496046 ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]


Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [8]:
from scipy.linalg import qr
qq, rr = qr(a)
print('Q from library', qq, 'R from library', rr, sep='\n')
assert_allclose(np.dot(qq, rr), a)

Q from library
[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]]
R from library
[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]


Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here* (10% of the total grade, peer-graded)
Два последних столбика q зануляются при перемножении q и r, а значит, неважно, что в них написано. 
Разница в знаках объясняется тем, что векторы могут быть противоположно направлены(при этом параллельны). Поэтому два разложения совпадают с точность до знака перед векторами(столбики в Q м строчки в R).

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [9]:
def a_to_RU(a):
    a1 = a.copy()
    m,n = a.shape
    U = np.zeros_like(a)

    for i in range(n if n<m else m-1):#Проверяем, какое измерение матрицы больше
        vec = a1[i:,i]#Вектор, который хотим повернуть - от диагонали и ниже
        y = np.zeros_like(vec)
        y[0] = np.linalg.norm(vec)
        u = np.zeros(m-i)#Наш reflection vector(вектор отражения?)
        u = (vec-y)/np.linalg.norm(vec-y)#Заполняем как в Хаусхолдере
        if abs(u[0])<1e-6:
            u[0] = -(sum(vec**2)-vec[0]**2)/(vec[0]+y[0])#Чтобы избежать зануления в случае, когда векторы почти параллельны
        a1[i:,i:] = a1[i:,i:] -  2 * np.dot(u.reshape(-1,1), (u @ a1[i:,i:]).reshape(1,-1))#Преобразование из инструкции
        
        U[i:,i] = u#Нужный столбик матрицы заполняем нашим вектором
    return a1, U

def aU_to_R(a, U):
    a1 = a.copy()
    m,n = a.shape
    R = a1.copy()

    for i in range(n if n<m else m-1):
        u = U[i:,i]#Берем i-тый вектор отражения
        R[i:,i:] = R[i:,i:] - 2 * np.dot(u.reshape(-1,1), (u @ R[i:,i:]).reshape(1,-1))#Находим (1-2u_i@u_i.T)a 
    return R

In [10]:
print("Test 1: vertical rectangular matrix")
a = rndm.uniform(size=(5,3))
R, U = a_to_RU(a)
print('a',a,'U', U,'R from the 1 function',R,'R from the 2 function', aU_to_R(a, U),
      'R from library',qr(a)[1],sep='\n')

print("Test 2: square matrix")
a = rndm.uniform(size=(5,5))
R, U = a_to_RU(a)
print('a',a,'U', U,'R from the 1 function',R,'R from the 2 function', aU_to_R(a, U),
      'R from library',qr(a)[1],sep='\n')

print("Test 3: horizontal rectangular matrix")
a = rndm.uniform(size=(3,5))
R, U = a_to_RU(a)
print('a',a,'U', U,'R from the 1 function',R,'R from the 2 function', aU_to_R(a, U),
      'R from library',qr(a)[1],sep='\n')

print("Test 4: bad matrix")
a = rndm.uniform(size=(5,3))
a[0,0] = 100
R, U = a_to_RU(a)
print('a',a,'U', U,'R from the 1 function',R,'R from the 2 function', aU_to_R(a, U),
      'R from library',qr(a)[1],sep='\n')

#И снова все совпадает с точностью до знака
#А значит, разложение верное

Test 1: vertical rectangular matrix
a
[[0.56119619 0.50308317 0.01376845]
 [0.77282662 0.88264119 0.36488598]
 [0.61539618 0.07538124 0.36882401]
 [0.9331401  0.65137814 0.39720258]
 [0.78873014 0.31683612 0.56809865]]
U
[[-0.57604006  0.          0.        ]
 [ 0.40205315 -0.30471224  0.        ]
 [ 0.32015198 -0.79045901 -0.78244581]
 [ 0.48545419  0.38872927 -0.62222608]
 [ 0.41032676 -0.3622355   0.0247639 ]]
R from the 1 function
[[ 1.66846046  1.11993758  0.80038785]
 [ 0.          0.55520169 -0.18167936]
 [ 0.         -0.          0.27611658]
 [ 0.          0.         -0.        ]
 [ 0.         -0.          0.        ]]
R from the 2 function
[[ 1.66846046  1.11993758  0.80038785]
 [ 0.          0.55520169 -0.18167936]
 [ 0.         -0.          0.27611658]
 [ 0.          0.         -0.        ]
 [ 0.         -0.          0.        ]]
R from library
[[-1.66846046 -1.11993758 -0.80038785]
 [ 0.         -0.55520169  0.18167936]
 [ 0.          0.         -0.27611658]
 [ 0.          